## Get Subset for Training

In [1]:
from datasets import load_dataset

ds = load_dataset('openpecha/tagged_cleaned_MT_v1.0.3', split='train')

In [2]:
def condition(example):
    return example['Tag'] != ''

ds = ds.filter(condition)

In [3]:
# Define the size of the random subset
subset_size = 50_000

# Shuffle the dataset
shuffled_ds = ds.shuffle(seed=0)  # Use a fixed seed for reproducibility

# Select the first `subset_size` examples
random_subset = shuffled_ds.select(range(subset_size))

In [4]:
random_subset = random_subset.train_test_split(.1)

In [5]:
random_subset

DatasetDict({
    train: Dataset({
        features: ['Source', 'Target', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'],
        num_rows: 45000
    })
    test: Dataset({
        features: ['Source', 'Target', 'File_Name', 'Machine Aligned', '__index_level_0__', 'Tag'],
        num_rows: 5000
    })
})

In [6]:
random_subset.save_to_disk('rat-poc-ds')

Saving the dataset (0/1 shards):   0%|          | 0/45000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]

## Add Contexts

Add similar sentences as context to mimic retrieval augmentation. The context for both train and eval come from the train set to mimic having a set of contexts from the training data.

In [7]:
from datasets import load_from_disk

ds = load_from_disk('rat-poc-ds')

In [8]:
from sentence_transformers import SentenceTransformer

# Load pre-trained embedding model
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Or a domain-specific model

# Encode all source sentences into vectors
sentences = ds['train']['Target']
embeddings = embedding_model.encode(sentences, convert_to_tensor=True)


/home/j/Documents/MLotsawa/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
import torch
from torch.nn.functional import cosine_similarity

def get_top_n_contexts(source_idx, source_embeddings, n=3):
    # Get the embedding of the target sentence
    query_embedding = source_embeddings[source_idx]

    # Compute cosine similarities with all other embeddings
    similarities = cosine_similarity(query_embedding.unsqueeze(0), source_embeddings)

    # Get the indices of the top-N most similar sentences (excluding itself)
    top_n_indices = torch.topk(similarities, n + 1).indices[1:]  # Skip the first (self)
    
    # Retrieve the corresponding sentences
    return list(top_n_indices)

# Example usage for the first sentence
context_idxs = get_top_n_contexts(0, embeddings, n=3)
print("Contexts for the first sentence:", context_idxs)

Contexts for the first sentence: [tensor(23107, device='cuda:0'), tensor(21350, device='cuda:0'), tensor(2325, device='cuda:0')]


In [10]:
# Add contexts to each example using `dataset.map`
def add_contexts(example, idx):
    # Call get_top_n_contexts with only necessary arguments
    context_idxs = get_top_n_contexts(idx, embeddings, n=3)
    
    # Construct the context by accessing the dataset using the indices
    example["context"] = [
        ds['train'][int(context_idx)]['Source'] + ' -> ' + ds['train'][int(context_idx)]['Target']
        for context_idx in context_idxs
    ]
    return example

# Apply the mapping
dataset_with_contexts = ds.map(
    add_contexts, 
    with_indices=True,  # Pass the index to `add_contexts`
    batched=False  # Process one example at a time
)


Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:
# Inspect the new dataset
print(dataset_with_contexts['train'][0])

{'Source': 'སྒོམ། སྤྱོད། འབྲས་ཀྱི་རྣམ་གཞག་ཅུང་ཟད་བརྗོད་ན། འཇུག་སྒོ་ནི། ཕྱི་ནང་སྒྲུབ་གསང་ཆུ་བོ་བཞི་རྫོགས་ཀྱི་དབང་སུམ་ཅུ་རྩ་དྲུག་གིས་རྒྱུད་སྨིན་ཅིང་། དམ་ཚིག་གཞུང་བཞིན་སྲུང་བའོ།། གཞི་ལྟ་བ་གཏན་ལ་འབེབས་ཚུལ་ནི། ཆོས་ཐམས་ཅད་ཀྱི་གཤིས་ལུགས་དཀྱིལ་འཁོར་གསུམ་དུ་གནས་པ་ཉིད་ཀྱི་ཆོས་ཅན་ཤེས་བྱའི་དོན་ལ་ཤེས་བྱེད་རིགས་པའི་གཏན་ཚིགས་རྣམས་ཀྱི་ཇི་ལྟ་བར་གཏན་ལ་ཕབ་སྟེ་རྟོགས་པའོ།། ལམ་སྒོམ་པས་ཉམས་སུ་ལེན་ཚུལ་ནི། ཆོས་ཉིད་སྙིང་པོ་ཇི་བཞིན་པའི་ངང་ལ་མཉམ་པར་འཇོག་པ་མཚན་མེད་རྣམ་པར་མི་རྟོག་པའི་ཏིང་ངེ་འཛིན་དང་། བསྐྱེད་སྔགས་བརྗོད་པ་ཙམ་གྱིས་རྟེན་དང་བརྟེན་པའི་འཁོར་ལོར་བསྒོམ་པ་མཚན་བཅས་ལྷའི་ཏིང་འཛིན་བསྒོམ་པ་གྲོལ་ལམ་དང་། ཐབས་ལམ་སྟེང་འོག་གི་སྒོ་ལ་བརྟེན་ནས་བདེ་སྟོང་གི་ཡེ་ཤེས་བསྐྱེད་པ་སྟེ་ལམ་གཉིས་ཀྱི་རྣལ་འབྱོར་བསྒོམ་པའོ།།', 'Target': 'Once again, let us say a little about its point of entry, view, meditation, conduct and results: i. Entry Point One’s mind is matured through the thirty-six empowerments in which the four rivers—outer, inner, accomplishing and secret—are complete, and one keeps the samayas as described in the texts. ii. 

In [12]:
dataset_with_contexts.save_to_disk('rat-poc-ds-w-context')

Saving the dataset (0/1 shards):   0%|          | 0/45000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000 [00:00<?, ? examples/s]